In [27]:
import config
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from IPython.display import display
import json
import datetime
import math
import joblib

from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score, v_measure_score, normalized_mutual_info_score

In [28]:
today = datetime.date.today()

proc_dir = config.PROC_DATA_PATH

folder = "2025-08-23_hdbscan"
folder_date = folder.split('_')[0]
rawq_runid = "run2"

img_path = config.SCHULTHESS_DATAPATH
# img_path = os.path.join(img_path, "600x600_imgs")

rawq_filepath = os.path.join(proc_dir, folder, "questionnaire", rawq_runid)
save_path = os.path.join(proc_dir, f'{today}_hdbscan_eval', 'rawq_radiographic')
os.makedirs(save_path, exist_ok=True)
rawq = pd.read_csv(os.path.join(rawq_filepath, f'questionnaire_{rawq_runid}_umap_hdbscan_scaled_wKL.csv'))
ids = pd.read_csv(os.path.join(rawq_filepath, "x_umap_ids.csv"))
display(rawq.head())

,id,cluster_label,probability,Unnamed: 0,record_id,visit,side,pain,age,ce_height,...,koos_a17,koos_sp1,koos_sp2,koos_sp3,koos_sp4,koos_sp5,koos_q1,koos_q2,koos_q3,koos_q4
0,IM0001_1_left,2,0.472182,0,IM0001,1,l,0.0,41,183.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,IM0002_1_left,0,1.000000,1,IM0002,1,l,0.0,40,175.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,IM0004_1_left,1,0.997089,2,IM0004,1,l,0.0,55,151.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,IM0005_1_left,2,0.686842,3,IM0005,1,l,0.0,46,184.5,...,0.0,0.0,2.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0
4,IM0008_1_left,2,0.927332,4,IM0008,1,l,0.0,51,166.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0


In [43]:
folder = "2025-09-13_hdbscan_img"
radiog_folder = folder
folder_date = folder.split('_')[0]
radiog_runid = "run32"

date = folder.split('_')[0]
radiographic_filepath = os.path.join(proc_dir, "radiographic_features", folder, radiog_runid)

df = pd.read_csv(os.path.join(radiographic_filepath, f'{folder}_{radiog_runid}_umap_hdbscan_scaled.csv'))

kl = os.path.join(proc_dir, "2025-08-11_data_exploration", "inmodi_data_questionnaire_kl_woSC.csv")#data/processed/2025-08-11_data_exploration/inmodi_data_questionnaire_kl_woSC.csv
kl = pd.read_csv(kl)

print(f"DF Shape before merge: {df.shape}")

radiog = df.merge(kl, left_on='id', right_on='name', how='inner')
print(f"DF Shape after merge: {radiog.shape}")
display(radiog.head())

DF Shape before merge: (427, 4)
DF Shape after merge: (382, 79)


,id,label,cluster_label,probability,Unnamed: 0,record_id,visit,side,pain,age,...,koos_a17,koos_sp1,koos_sp2,koos_sp3,koos_sp4,koos_sp5,koos_q1,koos_q2,koos_q3,koos_q4
0,IM1081_2_right,0,-1,0.0,347,IM1081,2,r,0.0,40,...,0.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
1,IM0010_2_right,0,-1,0.0,317,IM0010,2,r,0.0,46,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,IM0017_2_right,0,3,1.0,324,IM0017,2,r,0.0,37,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,IM1516_1_left,0,3,1.0,42,IM1516,1,l,0.0,59,...,1.0,0.0,0.0,0.0,1.0,4.0,3.0,3.0,2.0,3.0
4,IM1045_2_left,0,3,1.0,140,IM1045,2,l,0.0,42,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


# Cluster/KL-Score Comparison

In [30]:
print('Normalized Mutual Information Score:')

ground_truth = 'KL-Score'
label = 'cluster_label'

radiog_nmi = normalized_mutual_info_score(radiog[ground_truth], radiog[label], average_method = 'geometric')
print(f'Radiographic NMI: {radiog_nmi}')
rawq_nmi = normalized_mutual_info_score(rawq[ground_truth], rawq[label], average_method = 'geometric')
print(f'Rawq NMI: {rawq_nmi}')

Normalized Mutual Information Score:
Radiographic NMI: 0.057782196387909095
Rawq NMI: 0.1569146829204577


In [31]:
print('V-Measure Score:')
radiog_v_measure = v_measure_score(radiog[ground_truth], radiog[label])
print(f'Radiographic V-Measure: {radiog_v_measure}')
rawq_v_measure = v_measure_score(rawq[ground_truth], rawq[label])
print(f'Rawq V-Measure: {rawq_v_measure}')

V-Measure Score:
Radiographic V-Measure: 0.05745038055129523
Rawq V-Measure: 0.15648133642497525


# Cluster/Cluster Comparison

In [32]:
cols =['id', 'cluster_label', 'probability',  'record_id',
       'visit', 'side']

In [33]:
df_comb = rawq.merge(radiog[cols], on='id', suffixes=('_rawq', '_radiog'))


In [34]:
print('Normalized Mutual Information Score:')

ground_truth = 'cluster_label_rawq'
label = 'cluster_label_radiog'

radiog_nmi = normalized_mutual_info_score(df_comb[ground_truth], df_comb[label], average_method = 'geometric')
print(f'Combined NMI: {radiog_nmi}')

print('V-Measure Score:')
radiog_v_measure = v_measure_score(df_comb[ground_truth], df_comb[label])
print(f'Combined V-Measure: {radiog_v_measure}')

Normalized Mutual Information Score:
Combined NMI: 0.1080866340470964
V-Measure Score:
Combined V-Measure: 0.10637812477589251


# Silhouette Score

## RawQ

In [35]:
X_rawq = np.load(os.path.join(rawq_filepath, 'X_umap_embeddings.npy'), allow_pickle=True)
rawq_model = joblib.load(os.path.join(rawq_filepath, f'questionnaire_{rawq_runid}_umap_hdbscan_scaled_clusterer.pkl'))

In [37]:
print('RawQ Silhouette Score:')
rawq_silhouette = silhouette_score(X_rawq, rawq_model.fit_predict(X_rawq))
print(f'RawQ Silhouette Score: {rawq_silhouette}')


RawQ Silhouette Score:
RawQ Silhouette Score: 0.6837640404701233


## Radiographic

In [44]:
embeddings = np.load(os.path.join(radiographic_filepath, 'embeddings.npz'), allow_pickle=True)
X_radiog = embeddings['X_umap']
radiog_model = joblib.load(os.path.join(radiographic_filepath, f'{radiog_folder}_{radiog_runid}_umap_hdbscan_scaled_clusterer.pkl'))

In [45]:
print('Radiographic Silhouette Score:')
radiog_silhouette = silhouette_score(X_radiog, radiog_model.fit_predict(X_radiog))
print(f'Radiographic Silhouette Score: {radiog_silhouette}')

Radiographic Silhouette Score:
Radiographic Silhouette Score: 0.18941570818424225
